In [ ]:
%run CycleGAN_main.py --gpu 0 --datapath /root/work/Datasets/Day2Night --batch_size 1

Training epoch: 1 [0/2000 (0%)] | D loss (A): 0.878328 | D loss (B): 0.572918 | G loss: 19.443069 | Consistency: 1.311555 |
Training epoch: 1 [100/2000 (5%)] | D loss (A): 0.122086 | D loss (B): 0.312253 | G loss: 3.762844 | Consistency: 0.297104 |
Training epoch: 1 [200/2000 (10%)] | D loss (A): 0.216540 | D loss (B): 0.206244 | G loss: 4.162448 | Consistency: 0.327086 |
Training epoch: 1 [300/2000 (15%)] | D loss (A): 0.199673 | D loss (B): 0.162043 | G loss: 3.291887 | Consistency: 0.222593 |
Training epoch: 1 [400/2000 (20%)] | D loss (A): 0.124540 | D loss (B): 0.244573 | G loss: 5.320621 | Consistency: 0.454168 |
Training epoch: 1 [500/2000 (25%)] | D loss (A): 0.173469 | D loss (B): 0.186490 | G loss: 6.645848 | Consistency: 0.527969 |
Training epoch: 1 [600/2000 (30%)] | D loss (A): 0.085492 | D loss (B): 0.091554 | G loss: 5.467244 | Consistency: 0.452941 |
Training epoch: 1 [700/2000 (35%)] | D loss (A): 0.167641 | D loss (B): 0.454992 | G loss: 4.570864 | Consistency: 0.3778

In [1]:
%run LFSpix2pix_main.py --gpu 0 --batch_size 1 --train_path Data/day2night_pair.txt --test_path Data/day2night_testpair.txt

[3, 67, 195, 451, 963, 1475, 1987, 2499, 3011, 4035, 5059, 6083, 7107, 7619, 7875, 8003]
Training epoch: 1 [0/4000 (0%)] | D loss (A): 0.513233 | G loss: 59.035141 | L1: 0.587197 |
Training epoch: 1 [100/4000 (2%)] | D loss (A): 0.003873 | G loss: 83.298630 | L1: 0.823885 |
Training epoch: 1 [200/4000 (5%)] | D loss (A): 0.110476 | G loss: 74.665268 | L1: 0.737647 |
Training epoch: 1 [300/4000 (8%)] | D loss (A): 0.009502 | G loss: 78.830086 | L1: 0.779862 |
Training epoch: 1 [400/4000 (10%)] | D loss (A): 0.000697 | G loss: 74.127495 | L1: 0.731611 |
Training epoch: 1 [500/4000 (12%)] | D loss (A): 0.006447 | G loss: 52.879242 | L1: 0.519542 |
Training epoch: 1 [600/4000 (15%)] | D loss (A): 0.011663 | G loss: 56.338776 | L1: 0.554834 |
Training epoch: 1 [700/4000 (18%)] | D loss (A): 0.001091 | G loss: 56.283722 | L1: 0.553257 |
Training epoch: 1 [800/4000 (20%)] | D loss (A): 0.000602 | G loss: 73.237839 | L1: 0.722746 |
Training epoch: 1 [900/4000 (22%)] | D loss (A): 0.001286 | G 

FileNotFoundError: [Errno 2] No such file or directory: 'D'

In [15]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from CycleGAN_models import Generator

model_path_a2b = 'result/G_a2b'
model_path_b2a = 'result/G_b2a'
G_a2b = Generator(n_down=2, n_up=2, n_res=9, in_features=3).cuda()
G_a2b.load_state_dict(torch.load(model_path_a2b))
G_a2b.eval()
G_b2a = Generator(n_down=2, n_up=2, n_res=9, in_features=3).cuda()
G_b2a.load_state_dict(torch.load(model_path_b2a))
G_b2a.eval()

resize_img = transforms.Resize((256, 512))
to_tensor = transforms.ToTensor()
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
normalize = transforms.Normalize(mean=mean, std=std)
mean_ = torch.tensor(mean, dtype=torch.float32)[None,:,None,None]
std_ = torch.tensor(std, dtype=torch.float32)[None,:,None,None]
dataA_dir = '/root/work/Datasets/Day2Night/testA'
dataB_dir = '/root/work/Datasets/Day2Night/testB'
data_pix2pix_pair_AB = 'pix2pix_pair_AB_test'
data_pix2pix_pair_BA = 'pix2pix_pair_BA_test'
os.makedirs(data_pix2pix_pair_AB, exist_ok=True)
os.makedirs(data_pix2pix_pair_BA, exist_ok=True)
dataA_list = os.listdir(dataA_dir)
dataB_list = os.listdir(dataB_dir)

for idx, (itemA, itemB) in enumerate(zip(dataA_list, dataB_list)):
    dataA_path = os.path.join(dataA_dir, itemA)
    dataB_path = os.path.join(dataB_dir, itemB) 
    imgA = Image.open(dataA_path)
    imgB = Image.open(dataB_path)
    imgA_tensor = normalize(to_tensor(resize_img(imgA)))[None,:,:,:].cuda()
    imgB_tensor = normalize(to_tensor(resize_img(imgB)))[None,:,:,:].cuda()
    with torch.no_grad():
        fakeB = G_a2b(imgA_tensor)
        fakeA = G_b2a(imgB_tensor)
        fakeB_ = (fakeB.data.cpu() * std_) + mean_
        fakeA_ = (fakeA.data.cpu() * std_) + mean_
        #print(fakeA.shape)
    imgA_f = Image.fromarray((fakeA_ * 256.).clamp(min=0, max=255).squeeze().numpy().transpose(1,2,0).astype(np.uint8))
    imgB_f = Image.fromarray((fakeB_ * 256.).clamp(min=0, max=255).squeeze().data.cpu().numpy().transpose(1,2,0).astype(np.uint8))
    imgA.resize((512, 256)).save(os.path.join(data_pix2pix_pair_AB, 'img{:0>6}_real.jpg'.format(idx)))
    imgB_f.save(os.path.join(data_pix2pix_pair_AB, 'img{:0>6}_fake.jpg'.format(idx)))
    
    imgB.resize((512, 256)).save(os.path.join(data_pix2pix_pair_BA, 'img{:0>6}_real.jpg'.format(idx)))
    imgA_f.save(os.path.join(data_pix2pix_pair_BA, 'img{:0>6}_fake.jpg'.format(idx)))

IsADirectoryError: [Errno 21] Is a directory: '/root/work/Datasets/Day2Night/testA/.ipynb_checkpoints'

In [32]:
import shutil
txtpath = 'Data/night2day_testpair.txt'
if os.path.isdir(txtpath):
    os.remove(txtpath)
dirpath = 'Data/pix2pix_pair_BA_test'
itemlist = os.listdir(dirpath)
for idx in range(len(itemlist) // 2):
    real_path = os.path.join(dirpath, 'img{:0>6}_real.jpg'.format(idx))
    fake_path = os.path.join(dirpath, 'img{:0>6}_fake.jpg'.format(idx))
    
    with open(txtpath, 'a') as f:
        f.write(real_path + ' ' + real_path + ' ' + '0' + '\n')
        f.write(real_path + ' ' + fake_path + ' ' + '1' + '\n')

In [33]:
import random
with open(txtpath, 'r') as f:
    lines = f.readlines()
    
random.shuffle(lines)
torch.tensor(int(lines[0].split()[2]))

tensor(0)

In [10]:
import shutil
os.removedirs(os.path.join(dataA_dir, sorted(dataA_list)[0]))

FileNotFoundError: [Errno 2] No such file or directory: '/root/work/Datasets/Day2Night/trainA/.ipynb_checkpoints'

In [13]:
sorted(os.listdir(dataB_dir))

['Image00001.jpg',
 'Image00002.jpg',
 'Image00003.jpg',
 'Image00004.jpg',
 'Image00005.jpg',
 'Image00006.jpg',
 'Image00007.jpg',
 'Image00008.jpg',
 'Image00009.jpg',
 'Image00011.jpg',
 'Image00012.jpg',
 'Image00013.jpg',
 'Image00014.jpg',
 'Image00015.jpg',
 'Image00017.jpg',
 'Image00018.jpg',
 'Image00019.jpg',
 'Image00020.jpg',
 'Image00021.jpg',
 'Image00022.jpg',
 'Image00023.jpg',
 'Image00024.jpg',
 'Image00025.jpg',
 'Image00026.jpg',
 'Image00028.jpg',
 'Image00029.jpg',
 'Image00030.jpg',
 'Image00031.jpg',
 'Image00032.jpg',
 'Image00033.jpg',
 'Image00034.jpg',
 'Image00035.jpg',
 'Image00037.jpg',
 'Image00038.jpg',
 'Image00039.jpg',
 'Image00040.jpg',
 'Image00041.jpg',
 'Image00042.jpg',
 'Image00043.jpg',
 'Image00044.jpg',
 'Image00045.jpg',
 'Image00046.jpg',
 'Image00047.jpg',
 'Image00048.jpg',
 'Image00049.jpg',
 'Image00050.jpg',
 'Image00051.jpg',
 'Image00052.jpg',
 'Image00053.jpg',
 'Image00054.jpg',
 'Image00055.jpg',
 'Image00056.jpg',
 'Image00058